In [ ]:
import pandas as pd
import ipywidgets as widgets
import datetime as dt
from datetime import datetime
from IPython.display import display, clear_output
import matplotlib.pyplot as plt


In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
def exportarDados(dataframe):
    agora = datetime.now()
    export = agora.strftime('%Y-%m-%d_%H-%M-%S')
    export_df = dataframe.reset_index()
    export_df.to_excel(f"export/dataframe_exportado_{export}.xlsx", index=True)

In [ ]:
artigosPedido = pd.read_csv('data/artigosPedido.csv', sep=';')
desempenhoPraca = pd.read_csv('data/desempenhoPraca.csv', sep=';')
sacoleiras = pd.read_csv('data/sacoleiras.csv', sep=';')
pedidos = pd.read_csv('data/pedidos.csv', sep=';')
artigos = pd.read_csv('data/artigos.csv', sep=';')

In [ ]:
artigosPedido.head()

In [ ]:
artigosPedido.head()

In [ ]:
pedidos['status'].unique()

In [ ]:
def teste_seman_cobranca(vendedor):
    df = pedidos[pedidos['ano_cobranca'] > 2022]
    df = df[df['id_vendedor'] == vendedor]
    df = df[(df['status'].isin(['FECHADO', 'PEPINO']))]

    df = df[(df['status'].isin(['FECHADO']))]
    df['ano_sem_cobranca_data'] = pd.to_datetime(df['ano_sem_cobranca_data'])
    df.loc[:, 'parte_do_mes'] = df['ano_sem_cobranca_data'].dt.day / df['ano_sem_cobranca_data'].dt.days_in_month
    df.loc[:, 'parte_do_mes'] = pd.cut(df['parte_do_mes'], bins=[0, 0.25, 0.5, 0.75, 1.0], labels=['1a', '2a', '3a', '4a'])
    resumo_mensal = df.groupby(['parte_do_mes', df['ano_sem_cobranca_data'].dt.month, 'status' ]).agg({'valor_liquido': 'sum'}).reset_index()
    pivot = pd.pivot_table(df, values = 'valor_liquido', index = ['parte_do_mes', 'status'], aggfunc='sum')
    pivot2 = pd.pivot_table(df, values = 'valor_liquido', index = ['parte_do_mes', 'status'], aggfunc='sum')
    pivot2['porcentagem_do_total'] = pivot2.groupby(['status'])['valor_liquido'].transform(lambda x: (x / x.sum()) * 100)
    return pivot2

In [ ]:
teste_seman_cobranca(3)

In [ ]:
teste_seman_cobranca(17)

In [ ]:
teste_seman_cobranca(1)

In [ ]:
desempenhoPraca.columns

In [ ]:
artigosPedido.columns

In [ ]:
base_pedidos = pd.read_csv('data/pedidos.csv', sep=';')
base_pedidos.columns

In [ ]:
class MainApp:
    def __init__(self):
        self.load_data()
        self.create_layout()
        self._update_termos()    
        
    def filtrar(self, df):
        if df == 'artigosPedido':
            retorno = self.base_artigosPedido.query("ref in @self.filtro_artigos and @self.filtro_dataInicial <= ano_mes_cobranca <= @self.filtro_dataFinal and id_vendedor in @self.filtro_promotores and id_praca in @self.filtro_pracas ")
            retorno = pd.merge(retorno, self.base_artigos[['ref','artigo', 'descricao_grupo']], on='ref')
            retorno = pd.merge(retorno, self.base_pracas[['id_praca','descricao_praca']])
            retorno = pd.merge(retorno, self.base_vendedores[['id_vendedor', 'descricao_vendedor']], on='id_vendedor')
            
            return retorno
        
        elif df == 'pedidos':
            retorno = self.base_pedidos.query("@self.filtro_dataInicial <= ano_mes_cobranca <= @self.filtro_dataFinal and id_vendedor in @self.filtro_promotores and id_praca in @self.filtro_pracas ")
            retorno = pd.merge(retorno, self.base_pracas[['id_praca','descricao_praca']])
            retorno = pd.merge(retorno, self.base_vendedores[['id_vendedor', 'descricao_vendedor']], on='id_vendedor')
            retorno = pd.merge(retorno, self.base_sacoleiras[['id_sacoleira', 'cidade', 'nome', ]], on='id_sacoleira')
            return retorno
        
        elif df == 'desempenhos':
            retorno = self.base_desempenhoPraca.query("@self.filtro_dataInicial <= ano_mes_cobranca <= @self.filtro_dataFinal and id_vendedor in @self.filtro_promotores and id_praca in @self.filtro_pracas ")
        
        else:
            return 0
    
    def load_data(self):
        self.base_vendedores = pd.read_csv('data/vendedores.csv', sep=';')
        self.base_pracas = pd.read_csv('data/pracas.csv', sep=';')
        self.base_desempenhoPraca = pd.read_csv('data/desempenhoPraca.csv', sep=';')
        self.base_pedidos = pd.read_csv('data/pedidos.csv', sep=';')
        self.base_sacoleiras = pd.read_csv('data/sacoleiras.csv', sep=';')
        self.base_artigosPedido = pd.read_csv('data/artigosPedido.csv', sep=';')
        self.base_gruposArtigos = pd.read_csv('data/grupos.csv', sep=';')
        self.base_artigos = pd.read_csv('data/artigos.csv', sep=';')
        self.base_artigosPedido['ano_mes_cobranca'] = pd.to_datetime(self.base_artigosPedido['ano_mes_cobranca'], format='%m/%Y').dt.date
        self.base_desempenhoPraca['ano_mes_cobranca'] = pd.to_datetime(self.base_desempenhoPraca['ano_mes_cobranca'], format='%Y-%m').dt.date
        self.base_pedidos['ano_mes_cobranca'] = pd.to_datetime(self.base_pedidos['ano_mes_cobranca'], format='%m/%Y').dt.date
        
    def create_layout(self):
        # Promotor
        self.check_soPromotorEPracaAtivo = widgets.Checkbox(description="Só Ativas?", value=True)
        listaPromotores = self.get_lista_promotores()
        self.select_promotores = widgets.SelectMultiple(description='Promotor(es)', options=listaPromotores, value=[promotor[1] for promotor in listaPromotores])
        self.check_todosPromotores = widgets.Checkbox(description="Todos", value=False)
        self.box_promotores = widgets.HBox(children=[self.select_promotores, self.check_todosPromotores])
        
        # Praça
        listaPracas = self.get_lista_pracas()
        self.select_pracas = widgets.SelectMultiple(description='Praça(s)', options=listaPracas, value=[praca[1] for praca in listaPracas])
        self.check_todasPracas = widgets.Checkbox(description="Todas", value=False)
        self.box_pracas = widgets.HBox(children=[self.select_pracas, self.check_todasPracas])
        self.box_promotorPraca = widgets.HBox(children=[self.box_promotores, self.box_pracas, self.check_soPromotorEPracaAtivo])
        
        # Periodo
        self.dateP_dataInicial = widgets.DatePicker(description='Data Inicial Cobrança', value=dt.date.today() - dt.timedelta(days=2 * 30))
        self.dateP_dataFinal = widgets.DatePicker(description='Data Final Cobrança', value=dt.date.today() + dt.timedelta(days=2 * 30))
        self.box_periodo = widgets.HBox(children=[self.dateP_dataInicial, self.dateP_dataFinal])
        
        # Grupos e Artigos
        listaGrupos = self.get_lista_grupos()
        self.select_gruposArtigos = widgets.SelectMultiple(description='Grupos', options=listaGrupos, value=[grupo[1] for grupo in listaGrupos])
        self.check_todosGrupos = widgets.Checkbox(description='Todos', value=False)
        listaArtigos = self.get_lista_artigos()
        self.select_artigos = widgets.SelectMultiple(description='Artigos', options=listaArtigos, value=[artigo[1] for artigo in listaArtigos])
        self.check_todosArtigos = widgets.Checkbox(description='Todos', value=False)
        self.box_gruposArtigos = widgets.HBox(children=[self.select_gruposArtigos, self.check_todosGrupos, self.select_artigos, self.check_todosArtigos])

        self.filtros_app = [self.box_promotorPraca, self.box_periodo, self.box_gruposArtigos]
        self.box_filtros = widgets.VBox(children=self.filtros_app)
        self.display_app = widgets.HBox(children=[widgets.Output()])
        self.tab_apps = widgets.Tab(children=[self.display_app])
        self.box_fixos = [self.box_filtros]
        self.box_main = widgets.VBox(children=self.box_fixos)

        # Add observers to all widgets
        self.add_observer_to_widgets(self.box_filtros)

        # Display the nested boxes
        display(self.box_main)
    
    def get_lista_promotores(self):
        listaPromotores = []
        promotores = self.base_vendedores.sort_values(by='descricao_vendedor', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            promotores = promotores.query('status_vendedor == "ATIVO"')

        for _, linha in promotores.iterrows():
            tupla = (linha['descricao_vendedor'], linha['id_vendedor'])
            listaPromotores.append(tupla)
        return listaPromotores

    def get_lista_pracas(self):
        listaPracas = []
        promotores = self.select_promotores.value
        pracas = self.base_pracas.query('id_vendedor.isin(@promotores)')
        pracas = pracas.sort_values(by='descricao_praca', ascending=True)

        if self.check_soPromotorEPracaAtivo.value:
            pracas = pracas.query('status_praca == "ATIVO"')

        for _, linha in pracas.iterrows():
            tupla = (linha['descricao_praca'], linha['id_praca'])
            listaPracas.append(tupla)

        return listaPracas

    def get_lista_grupos(self):
        listaGrupos = []
        grupos = self.base_gruposArtigos
        grupos = grupos.sort_values(by='descricao_grupo', ascending=True)

        for _, linha in grupos.iterrows():
            tupla = (linha['descricao_grupo'], linha['id_grupo'])
            listaGrupos.append(tupla)

        return listaGrupos

    def get_lista_artigos(self):
        listaArtigos = []
        grupos = self.select_gruposArtigos.value
        artigos = self.base_artigos.query('id_grupo.isin(@grupos)')
        artigos = artigos.sort_values(by='artigo', ascending=True)

        if self.check_soPromotorEPracaAtivo.value:
            artigos = artigos.query('status_artigo == "ATIVO"')

        for _, linha in artigos.iterrows():
            tupla = (linha['artigo'], linha['ref'])
            listaArtigos.append(tupla)

        return listaArtigos

    def add_observer_to_widgets(self, widget):
        # Specific functions
        self.select_promotores.observe(self._promotor_selecionado, names='value') # type: ignore
        self.select_gruposArtigos.observe(self._grupos_selecionados, names='value') # type: ignore
        self.check_todosPromotores.observe(self._selecionar_todos_promotores, names='value') # type: ignore
        self.check_todasPracas.observe(self._selecionar_todas_pracas, names='value') # type: ignore
        self.check_todosGrupos.observe(self._selecionar_todos_grupos, names='value') # type: ignore
        self.check_todosArtigos.observe(self._selecionar_todos_artigos, names='value') # type: ignore

        # General function
        if isinstance(widget, (widgets.Box, widgets.VBox, widgets.HBox, widgets.GridBox)):
            for child_widget in widget.children: # type: ignore
                self.add_observer_to_widgets(child_widget)
        elif isinstance(widget, widgets.Widget):
            widget.observe(self.normal_object_updated, names='value') # type: ignore

    def normal_object_updated(self, change):
        if change['new'] == ():
            pass
        else:
            self._update_termos()
                
    def _update_termos(self):
        # Convertendo as strings das datas para o formato "mm/aaaa"
        data_inicial_str = self.dateP_dataInicial.value.strftime("%m/%Y") # type: ignore
        data_final_str = self.dateP_dataFinal.value.strftime("%m/%Y") # type: ignore

        # Convertendo as strings para o tipo datetime
        data_inicial = datetime.strptime(data_inicial_str, "%m/%Y").date()
        data_final = datetime.strptime(data_final_str, "%m/%Y").date()

        self.filtro_promotores = self.select_promotores.value
        self.filtro_pracas= self.select_pracas.value
        self.filtro_dataInicial = data_inicial
        self.filtro_dataFinal= data_final
        self.filtro_artigos = self.select_artigos.value

    def set_funcao(self):
        funcao_selecionada = getattr(self.select_app.value, self.select_funcao.value) # type: ignore
        funcao_selecionada()

    def _updateOptionsPracas(self):
        self.select_pracas.options = self.get_lista_pracas()

    def _updateOptionsArtigos(self):
        self.select_artigos.options = self.get_lista_artigos()
        self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]

    def _promotor_selecionado(self, change):
        self._updateOptionsPracas()

    def _grupos_selecionados(self, change):
        self._updateOptionsArtigos()

    def _selecionar_todos_promotores(self, change):
        if change['new']:
            self.select_promotores.value = [promotor[1] for promotor in self.get_lista_promotores()]
        else:
            self.select_promotores.value = []

    def _selecionar_todas_pracas(self, change):
        if change['new']:
            self.select_pracas.value = [praca[1] for praca in self.get_lista_pracas()]
        else:
            self.select_pracas.value = []

    def _selecionar_todos_grupos(self, change):
        if change['new']:
            self.select_gruposArtigos.value = [grupo[1] for grupo in self.get_lista_grupos()]
        else:
            self.select_gruposArtigos.value = []

    def _selecionar_todos_artigos(self, change):
        if change['new']:
            self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]
        else:
            self.select_artigos.value = []


In [ ]:
base = MainApp()

In [ ]:
artigosPedido = base.filtrar('desempenhos')

In [ ]:
artigosPedido.head()

In [ ]:
group = artigosPedido.groupby(by'')